In [ ]:
import os 
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import keras 
from keras.callbacks import EarlyStopping,ModelCheckpoint
import tensorflow as tf
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from tqdm import tqdm
from imblearn.over_sampling import SMOTE

In [ ]:
images = []
labels = []
for subfolder in tqdm(os.listdir('/kaggle/input/ad-disease/train')):
    subfolder_path = os.path.join('/kaggle/input/ad-disease/train', subfolder)
    for folder in os.listdir(subfolder_path):
        subfolder_path2=os.path.join(subfolder_path,folder)
        for image_filename in os.listdir(subfolder_path2):
            image_path = os.path.join(subfolder_path2, image_filename)
            images.append(image_path)
            labels.append(folder)
df = pd.DataFrame({'image': images, 'label': labels})
df

In [ ]:
plt.figure(figsize=(5, 5))
class_counts = df['label'].value_counts()
labels = class_counts.index
sizes = class_counts.values

plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90, colors=sns.color_palette('Set1'))
plt.title('Distribution of Classes', fontsize=20)
plt.show()

In [ ]:
plt.figure(figsize=(50,50))
for n,i in enumerate(np.random.randint(0,len(df),20)):
    plt.subplot(10,5,n+1)
    img=cv2.imread(df.image[i])
    img=cv2.resize(img,(224,224))
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    plt.axis('off')
    plt.title(df.label[i],fontsize=25)

In [ ]:
class_num=np.sort(['MildDemented','ModerateDemented','NonDemented','VeryMildDemented'])
class_num

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
Size=(128,128)
work_dr = ImageDataGenerator(
    rescale = 1./255
)
train_data_gen = work_dr.flow_from_dataframe(df,x_col='image',y_col='label', target_size=Size, batch_size=6500, shuffle=False)

In [ ]:
# Get the first batch of data and labels
train_data, train_labels = train_data_gen[0]
for i in range(len(train_data_gen)):
    train_data, train_labels = train_data_gen[i]

In [ ]:
class_num=np.sort(['MildDemented','ModerateDemented','NonDemented','VeryMildDemented'])
class_num

In [ ]:
sm = SMOTE(random_state=42)
train_data, train_labels = sm.fit_resample(train_data.reshape(-1, 128 * 128 * 3), train_labels)
train_data = train_data.reshape(-1, 128,128, 3)
print(train_data.shape, train_labels.shape)

In [ ]:
labels=[class_num[i] for i in np.argmax(train_labels,axis=1) ]
plt.figure(figsize=(6,4))
ax = sns.countplot(x=labels,palette='Set1')
ax.set_xlabel("Class",fontsize=20)
ax.set_ylabel("Count",fontsize=20)
plt.title('The Number Of Samples For Each Class',fontsize=20)
plt.grid(True)
plt.xticks(rotation=45)
plt.show()

In [ ]:
X_train, X_test1, y_train, y_test1 = train_test_split(train_data,train_labels, test_size=0.3, random_state=42,shuffle=True,stratify=train_labels)
X_val, X_test, y_val, y_test = train_test_split(X_test1,y_test1, test_size=0.5, random_state=42,shuffle=True,stratify=y_test1)
print('X_train shape is ' , X_train.shape)
print('X_test shape is ' , X_test.shape)
print('X_val shape is ' , X_val.shape)
print('y_train shape is ' , y_train.shape)
print('y_test shape is ' , y_test.shape)
print('y_val shape is ' , y_val.shape)

In [ ]:
from keras.layers import Conv2D,Flatten,Dense,Dropout,BatchNormalization,MaxPooling2D

In [ ]:
from keras.models import Sequential

In [ ]:
model = Sequential()

model.add(Conv2D(32,kernel_size =(2,2),input_shape = (128,128,3),padding = 'Same'))
model.add(Conv2D(32,kernel_size =(2,2),activation='relu',padding = 'Same'))

model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,kernel_size =(2,2),activation='relu',padding = 'Same'))
model.add(Conv2D(64,kernel_size =(2,2),activation='relu',padding = 'Same'))

model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides = (2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
          
model.add(Dense(512,activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(4,activation='softmax'))
          
model.compile(loss = 'categorical_crossentropy',optimizer = 'Adamax',metrics=['accuracy'])
          
print(model.summary())

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
history = model.fit(X_train,y_train,epochs = 30,batch_size=10,verbose=1,validation_data=(X_test,y_test))

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Loss')
plt.ylabel('Epoch')
plt.legend(['Test','Validation'],loc='upper right')
plt.show()

In [ ]:
score, acc= model.evaluate(X_test,y_test)
print('Test Loss =', score)
print('Test Accuracy =', acc)

In [ ]:
y_pred = np.argmax(model.predict(X_test),axis=1)
print(np.mean(y_pred == np.argmax(y_test,axis=1)))

In [ ]:
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
confusion_mtx = confusion_matrix(np.argmax(y_test, axis=1), y_pred_classes)
print("Confusion Matrix:\n", confusion_mtx)

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
f1_scores = f1_score(np.argmax(y_test, axis=1), y_pred_classes, average='weighted')
print("F1 Score:", f1_scores)

In [ ]:
from sklearn.metrics import precision_score
precision_scores = precision_score(np.argmax(y_test, axis=1), y_pred_classes, average='weighted')
print("Precision:", precision_scores)

In [ ]:
from sklearn.metrics import recall_score
recall_scores = recall_score(np.argmax(y_test, axis=1), y_pred_classes, average='weighted')
print("Recall:", recall_scores)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import cohen_kappa_score
import joblib

In [ ]:
# Save the model
model_filename = 'trained_model.h5'
model.save(model_filename)
print(f'Model saved to {model_filename}')

In [ ]:
import numpy as np
if not isinstance(y_test, np.ndarray):
    y_test = np.array(y_test)

if not isinstance(y_pred, np.ndarray):
    y_pred = np.array(y_pred)

# Flatten to 1D array if necessary
if y_test.ndim > 1:
    y_test = y_test.ravel()

if y_pred.ndim > 1:
    y_pred = y_pred.ravel()

In [ ]:
# Create DataFrame with actual and predicted values
predictions = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
predictions_filename = 'predictions.csv'
predictions.to_csv(predictions_filename, index=False)
print(f'Predictions saved to {predictions_filename}')